### **3.31 - Wrap up!**

<center><img src="https://i.postimg.cc/4yzVkJXc/a310.png"></center>

In [ ]:

1.	from airflow import DAG
2.	from airflow.providers.postgres.operators.postgres import PostgresOperator
3.	from airflow.providers.http.sensors.http import <center><img src="httpsensor
4.	from airflow.providers.http.operators.http import SimpleHttpOperator
5.	from airflow.operators.python import PythonOperator
6.	from airflow.providers.postgres.hooks.postgres import PostgresHook
7.	 
8.	import json
9.	from pandas import json_normalize
10.	from datetime import datetime
11.	 
12.	def _process_user(ti):
13.	    user = ti.xcom_pull(task_ids="extract_user")
14.	    user = user['results'][0]
15.	    processed_user = json_normalize({
16.	        'firstname': user['name']['first'],
17.	        'lastname': user['name']['last'],
18.	        'country': user['location']['country'],
19.	        'username': user['login']['username'],
20.	        'password': user['login']['password'],
21.	        'email': user['email'] })
22.	    processed_user.to_csv('/tmp/processed_user.csv', index=None, header=False)
23.	 
24.	def _store_user():
25.	    hook = PostgresHook(postgres_conn_id='postgres')
26.	    hook.copy_expert(
27.	        sql="COPY users FROM stdin WITH DELIMITER as ','",
28.	        filename='/tmp/processed_user.csv'
29.	    )
30.	 
31.	with DAG('user_processing', start_date=datetime(2022, 1, 1), 
32.	        schedule_interval='@daily', catchup=False) as dag:
33.	 
34.	    create_table = PostgresOperator(
35.	        task_id='create_table',
36.	        postgres_conn_id='postgres',
37.	        sql='''
38.	            CREATE TABLE IF NOT EXISTS users (
39.	                firstname TEXT NOT NULL,
40.	                lastname TEXT NOT NULL,
41.	                country TEXT NOT NULL,
42.	                username TEXT NOT NULL,
43.	                password TEXT NOT NULL,
44.	                email TEXT NOT NULL
45.	            );
46.	        '''
47.	    )
48.	 
49.	    is_api_available = <center><img src="httpsensor(
50.	        task_id='is_api_available',
51.	        http_conn_id='user_api',
52.	        endpoint='api/'
53.	    )
54.	 
55.	    extract_user = SimpleHttpOperator(
56.	        task_id='extract_user',
57.	        http_conn_id='user_api',
58.	        endpoint='api/',
59.	        method='GET',
60.	        response_filter=lambda response: json.loads(response.text),
61.	        log_response=True
62.	    )
63.	 
64.	    process_user = PythonOperator(
65.	        task_id='process_user',
66.	        python_callable=_process_user
67.	    )
68.	 
69.	    store_user = PythonOperator(
70.	        task_id='store_user',
71.	        python_callable=_store_user
72.	    )
73.	 
74.	    create_table >> is_api_available >> extract_user >> process_user >> store_user


<center><img src="https://i.postimg.cc/BvkKj86D/a311.png"></center>
<center><img src="https://i.postimg.cc/pTjnz2hS/a312.png"></center>
<center><img src="https://i.postimg.cc/90jwBvyQ/a313.png"></center>